In [ ]:
import zipfile
import os

zip_file_path = '/content/drive/MyDrive/ClimateProject/NORESM2_LM_full.zip'
extraction_dir = '/tmp/NORESM2_LM_full'

os.makedirs(extraction_dir, exist_ok=True)

print(f"Extracting '{zip_file_path}' to '{extraction_dir}'...")
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extraction_dir)
print("Extraction complete.")
print(f"Contents of '{extraction_dir}':")
print(os.listdir(extraction_dir))

In [ ]:
print(f"Listing contents of '{extraction_dir}' recursively:")
for root, dirs, files in os.walk(extraction_dir):
    for d in dirs:
        print(os.path.join(root, d))
    for f in files:
        print(os.path.join(root, f))

In [ ]:
import glob, xarray as xr

EXTRACT_DIR = "/tmp/NORESM2_LM_full"

# Find all NetCDF files
all_nc = [f.replace("\\", "/") for f in glob.glob(f"{EXTRACT_DIR}/**/*.nc", recursive=True)]
print("Total NetCDF files found:", len(all_nc))

# Select Input4MIPs emissions (CO2 preferred)
input_files = [f for f in all_nc if any(v in f.lower() for v in ["co2", "input4mips", "emission"])]
# Select CMIP6 climate outputs (tas = temperature)
output_files = [f for f in all_nc if "tas" in f.lower() and "ssp" in f.lower()]

print(f"Input4MIPs candidates: {len(input_files)}")
print(f"CMIP6 candidates: {len(output_files)}")

# Automatically pick the first valid files
input_path = input_files[0] if input_files else None
output_path = output_files[0] if output_files else None

print("✅ Selected Input4MIPs file:", input_path)
print("✅ Selected CMIP6 file:", output_path)


In [ ]:
import glob, xarray as xr

EXTRACT_DIR = "/tmp/NORESM2_LM_full"

# Find all NetCDF files (replace backslashes in paths)
all_nc = [f.replace("\\", "/") for f in glob.glob(f"{EXTRACT_DIR}/**/*.nc", recursive=True)]
print("Total .nc files found:", len(all_nc))

# Pick one Input4MIPs CO2 file (preferably SSP126)
input_files = [f for f in all_nc if "ssp126" in f.lower() and "co2" in f.lower()]
# Pick one CMIP6 temperature (tas) file (same scenario SSP126)
output_files = [f for f in all_nc if "ssp126" in f.lower() and "tas" in f.lower()]

print("Input4MIPs CO2 candidates:", len(input_files))
print("CMIP6 tas candidates:", len(output_files))

input_path = input_files[0]
output_path = output_files[0]

print("✅ Using Input file:", input_path)
print("✅ Using Output file:", output_path)


In [ ]:
import glob, xarray as xr, os

EXTRACT_DIR = "/tmp/NORESM2_LM_full"

# Replace all backslashes in paths (so Colab can read them)
all_nc = []
for root, dirs, files in os.walk(EXTRACT_DIR):
    for name in files:
        if name.endswith(".nc"):
            full_path = os.path.join(root, name).replace("\\", "/")
            all_nc.append(full_path)

print("✅ Total .nc files found:", len(all_nc))

# Now safely filter based on names
input_candidates = [f for f in all_nc if "ssp126" in f.lower() and "co2" in f.lower()]
output_candidates = [f for f in all_nc if "ssp126" in f.lower() and "tas" in f.lower()]

print("Input candidates found:", len(input_candidates))
print("Output candidates found:", len(output_candidates))

# Let's preview the first few
print("🟢 Input example:", input_candidates[:3])
print("🟢 Output example:", output_candidates[:3])


In [ ]:
import os
count = 0
for root, dirs, files in os.walk("/tmp/NORESM2_LM_full"):
    for f in files:
        if f.endswith(".nc"):
            print(f)
            count += 1
            if count >= 10:
                break
    if count >= 10:
        break


In [ ]:
import os

base = "/tmp/NORESM2_LM_full"
fixed = 0

for root, dirs, files in os.walk(base):
    for f in files:
        if "\\" in f:
            new_name = f.replace("\\", "_")
            os.rename(os.path.join(root, f), os.path.join(root, new_name))
            fixed += 1

print(f"✅ Renamed {fixed} files that contained backslashes in their names.")


In [ ]:
import glob

files = glob.glob("/tmp/NORESM2_LM_full/**/*.nc", recursive=True)
print("Total .nc files after rename:", len(files))
print("Example files:")
for f in files[:10]:
    print(f)


In [ ]:
input_path = [f for f in files if "ssp126" in f and "co2" in f.lower()][0]
output_path = [f for f in files if "ssp126" in f and "tas" in f.lower()][0]

print("Input file:", input_path)
print("Output file:", output_path)


In [ ]:
import xarray as xr

input_path = [f for f in files if "ssp126" in f and "co2" in f.lower()][0]
output_path = [f for f in files if "ssp126" in f and "tas" in f.lower()][0]

print("🟢 Input file:", input_path)
print("🟢 Output file:", output_path)

# Open with decode_times=False to avoid calendar issue
ds_in  = xr.open_dataset(input_path, decode_times=False)
ds_out = xr.open_dataset(output_path, decode_times=False)

print("✅ Both files opened successfully (times not yet decoded).")
print("Input vars:", list(ds_in.data_vars))
print("Output vars:", list(ds_out.data_vars))


In [ ]:
import pandas as pd
import numpy as np

# Fix Input4MIPs time axis
if "time" in ds_in:
    start_year = 2015  # approximate since all are monthly 2015–2100
    ds_in["time"] = pd.date_range(f"{start_year}-01-01", periods=ds_in.time.size, freq="MS")

# Fix CMIP6 output time axis
if "time" in ds_out:
    start_year = 2015
    ds_out["time"] = pd.date_range(f"{start_year}-01-01", periods=ds_out.time.size, freq="MS")

print("✅ Time variables fixed and converted to standard datetime.")


**4 in 2 out**

In [ ]:
import os, glob, xarray as xr

BASE = "/tmp/NORESM2_LM_full"

# --- rename any files that still contain "\" ---
fixed = 0
for root, _, files in os.walk(BASE):
    for f in files:
        if "\\" in f:
            os.rename(os.path.join(root, f),
                      os.path.join(root, f.replace("\\", "_")))
            fixed += 1
print(f"✅ Renamed {fixed} files containing backslashes.")

# --- collect all .nc files ---
all_nc = glob.glob(f"{BASE}/**/*.nc", recursive=True)

# --- find 4 inputs + 2 outputs ---
inputs = {v: [f for f in all_nc if "ssp126" in f.lower() and v in f.lower()]
          for v in ["co2","ch4","so2","bc"]}
outputs = {v: [f for f in all_nc if "ssp126" in f.lower() and v in f.lower()]
           for v in ["tas","pr"]}

for k,v in inputs.items():  print(f"{k.upper()} files: {len(v)}")
for k,v in outputs.items(): print(f"{k.upper()} files: {len(v)}")

# pick one file per variable
paths_in  = {k:v[0] for k,v in inputs.items()  if v}
paths_out = {k:v[0] for k,v in outputs.items() if v}


In [ ]:
import pandas as pd, numpy as np

def open_fix_time(path, start_year=2015):
    ds = xr.open_dataset(path, decode_times=False)
    if "time" in ds:
        ds["time"] = pd.date_range(f"{start_year}-01-01",
                                   periods=ds.time.size, freq="MS")
    return ds

ds_inputs  = {k: open_fix_time(p) for k,p in paths_in.items()}
ds_outputs = {k: open_fix_time(p) for k,p in paths_out.items()}
print("✅ All 6 datasets opened and time fixed.")


In [ ]:
# Emission units
for k,ds in ds_inputs.items():
    for v in ds.data_vars:
        ds[v].attrs["units"] = "kg m-2 s-1"

# Temperature Kelvin→°C
if "tas" in ds_outputs:
    if ds_outputs["tas"]["tas"].attrs.get("units","").lower() in ["k","kelvin"]:
        ds_outputs["tas"]["tas"] = ds_outputs["tas"]["tas"] - 273.15
        ds_outputs["tas"]["tas"].attrs["units"] = "degC"

print("✅ Units normalized.")


In [ ]:
# choose reference grid (Temperature)
ref = ds_outputs["tas"]

for k,ds in ds_inputs.items():
    ds_inputs[k] = ds.sel(time=slice("2015-01-01","2025-12-31"))
    if (ds.lat.size != ref.lat.size) or (ds.lon.size != ref.lon.size):
        ds_inputs[k] = ds_inputs[k].interp(lat=ref.lat, lon=ref.lon)

for k,ds in ds_outputs.items():
    ds_outputs[k] = ds.sel(time=slice("2015-01-01","2025-12-31"))

print("✅ Temporal slice & grid alignment done.")


In [ ]:
data_vars = {}

# 4 inputs
for name,ds in ds_inputs.items():
    v = list(ds.data_vars)[0]
    data_vars[name.upper()] = ds[v]

# 2 outputs
for name,ds in ds_outputs.items():
    v = list(ds.data_vars)[0]
    rename = "Temperature" if name=="tas" else "Precipitation"
    data_vars[rename] = ds[v]

combined = xr.Dataset(data_vars).dropna(dim="time", how="all")

combined.attrs = {
    "source": "NorESM2-LM",
    "scenario": "SSP126",
    "inputs": "CO2,CH4,SO2,BC",
    "outputs": "Temperature,Precipitation",
    "years": "2015–2025",
    "temporal_res": "monthly",
    "spatial_res": "≈250 km",
    "created_by": "Richee"
}
print("✅ Unified 4 input + 2 output dataset built.")
combined


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/ClimateProject/NORESM2_LM_2015_2025_4in2out.nc"
combined.to_netcdf(SAVE_PATH)
print("✅ Saved:", SAVE_PATH)


In [ ]:
import matplotlib.pyplot as plt

combined["Temperature"].isel(time=0).plot()
plt.title("Global Temperature (°C) — Jan 2015 (SSP126)")
plt.show()

combined["CO2"].isel(time=0).plot()
plt.title("CO₂ Emissions (kg m⁻² s⁻¹) — Jan 2015 (SSP126)")
plt.show()


In [ ]:
# Check time coverage for each dataset
def show_time_info(label, ds):
    if "time" in ds:
        t0 = str(ds.time.values[0])[:10]
        t1 = str(ds.time.values[-1])[:10]
        print(f"{label} → {t0}  to  {t1}  ({ds.time.size} time steps)")
    else:
        print(f"{label} → ⚠️ No 'time' dimension found")

for k, ds in ds_inputs.items():
    show_time_info(f"Input {k.upper()}", ds)

for k, ds in ds_outputs.items():
    show_time_info(f"Output {k.upper()}", ds)


In [ ]:
# Subset to desired window
start_year, end_year = 2015, 2025

for k in ds_inputs:
    if "time" in ds_inputs[k]:
        ds_inputs[k] = ds_inputs[k].sel(time=slice(f"{start_year}-01-01", f"{end_year}-12-31"))

for k in ds_outputs:
    if "time" in ds_outputs[k]:
        ds_outputs[k] = ds_outputs[k].sel(time=slice(f"{start_year}-01-01", f"{end_year}-12-31"))

print(f"✅ Datasets trimmed to {start_year}–{end_year}.")


In [ ]:
for k, ds in ds_inputs.items():
    show_time_info(f"Trimmed Input {k.upper()}", ds)

for k, ds in ds_outputs.items():
    show_time_info(f"Trimmed Output {k.upper()}", ds)


In [ ]:
import glob

def get_yearly_files(variable):
    paths = sorted([f for f in all_nc
                    if "ssp126" in f.lower() and variable.lower() in f.lower()
                    and any(str(y) in f for y in range(2015, 2026))])
    print(f"{variable.upper()}: {len(paths)} yearly files found.")
    return paths

co2_files = get_yearly_files("co2")
ch4_files = get_yearly_files("ch4")
so2_files = get_yearly_files("so2")
bc_files  = get_yearly_files("bc")
tas_files = get_yearly_files("tas")
pr_files  = get_yearly_files("pr")


In [ ]:
def open_concat(paths):
    ds = xr.open_mfdataset(paths, combine="by_coords", decode_times=False)
    ds["time"] = pd.date_range("2015-01-01", periods=ds.time.size, freq="MS")
    return ds

ds_in = {
    "co2": open_concat(co2_files),
    "ch4": open_concat(ch4_files),
    "so2": open_concat(so2_files),
    "bc" : open_concat(bc_files)
}

ds_out = {
    "tas": open_concat(tas_files),
    "pr" : open_concat(pr_files)
}

print("✅ All yearly files concatenated (2015–2025).")


In [ ]:
for k, ds in ds_in.items():
    print(f"Input {k.upper()} → {str(ds.time.values[0])[:10]} to {str(ds.time.values[-1])[:10]} ({ds.time.size} steps)")
for k, ds in ds_out.items():
    print(f"Output {k.upper()} → {str(ds.time.values[0])[:10]} to {str(ds.time.values[-1])[:10]} ({ds.time.size} steps)")


In [ ]:
# Kelvin → Celsius for temperature
if "tas" in ds_out:
    if ds_out["tas"]["tas"].attrs.get("units", "").lower() in ["k", "kelvin"]:
        ds_out["tas"]["tas"] = ds_out["tas"]["tas"] - 273.15
        ds_out["tas"]["tas"].attrs["units"] = "degC"

# Standardize input emission units
for k, ds in ds_in.items():
    for v in ds.data_vars:
        ds[v].attrs["units"] = "kg m-2 s-1"

# Choose reference grid (Temperature)
ref = ds_out["tas"]

# Align all inputs spatially and temporally
for k in ds_in:
    ds_in[k] = ds_in[k].sel(time=slice("2015-01-01", "2025-12-31"))
    if (ds_in[k].lat.size != ref.lat.size) or (ds_in[k].lon.size != ref.lon.size):
        ds_in[k] = ds_in[k].interp(lat=ref.lat, lon=ref.lon)

# Align outputs as well
for k in ds_out:
    ds_out[k] = ds_out[k].sel(time=slice("2015-01-01", "2025-12-31"))

print("✅ Units normalized & grids aligned across all 6 variables.")


In [ ]:
import xarray as xr

data_vars = {}

# 4 Inputs
for k, ds in ds_in.items():
    v = list(ds.data_vars)[0]
    data_vars[k.upper()] = ds[v]

# 2 Outputs
data_vars["Temperature"]    = ds_out["tas"]["tas"]
data_vars["Precipitation"]  = ds_out["pr"][list(ds_out["pr"].data_vars)[0]]

# Combine all into one dataset
combined = xr.Dataset(data_vars).dropna(dim="time", how="all")

# Metadata
combined.attrs = {
    "source": "NorESM2-LM",
    "scenario": "SSP126",
    "inputs": "CO2, CH4, SO2, BC",
    "outputs": "Temperature, Precipitation",
    "years": "2015–2025",
    "year_start": 2015,
    "year_end": 2025,
    "temporal_res": "monthly",
    "spatial_res": "≈250 km",
    "created_by": "Richee",
    "pipeline_stages": "Downloader → Checker → RawProcessor → ResolutionProcessor → StructureProcessor"
}

print("✅ Unified 4-input, 2-output dataset created successfully.")
combined


In [ ]:
SAVE_PATH = "/content/drive/MyDrive/ClimateProject/NORESM2_LM_2015_2025_4in2out.nc"
combined.to_netcdf(SAVE_PATH)
print("✅ Saved final ML-ready dataset at:", SAVE_PATH)


In [ ]:
print("Variables:", list(combined.data_vars))
print("Dimensions:", combined.dims)
print("Time span:", str(combined.time.values[0])[:10], "to", str(combined.time.values[-1])[:10])


In [ ]:
import matplotlib.pyplot as plt

# Plot 1 — Surface temperature
combined["Temperature"].isel(time=0).plot()
plt.title("🌡️ Global Temperature (°C) — Jan 2015")
plt.show()

# Plot 2 — CO₂ emissions
combined["CO2"].isel(time=0).plot()
plt.title("💨 CO₂ Emissions (kg m⁻² s⁻¹) — Jan 2015")
plt.show()

# Plot 3 — Precipitation
combined["Precipitation"].isel(time=0).plot()
plt.title("🌧️ Precipitation (mm/day) — Jan 2015")
plt.show()


**Stage 6 Preparing for ML ready **

In [ ]:
import xarray as xr
import numpy as np

path = "/content/drive/MyDrive/ClimateProject/NORESM2_LM_2015_2025_4in2out.nc"
ds = xr.open_dataset(path)
print("✅ Loaded ClimateSet mini-dataset.")
print("Variables:", list(ds.data_vars))
print("Dimensions:", ds.dims)


In [ ]:
input_vars  = ["CO2", "CH4", "SO2", "BC"]
output_vars = ["Temperature", "Precipitation"]

# Stack into NumPy arrays
X = np.stack([ds[v].values for v in input_vars], axis=1)  # shape = [time, 4, y, x]
Y = np.stack([ds[v].values for v in output_vars], axis=1) # shape = [time, 2, y, x]

print("X shape (inputs):", X.shape)
print("Y shape (outputs):", Y.shape)


In [ ]:
# Min–max normalization across each variable
X_norm = np.empty_like(X)
Y_norm = np.empty_like(Y)

for i in range(X.shape[1]):
    vmin, vmax = np.nanmin(X[:, i, :, :]), np.nanmax(X[:, i, :, :])
    X_norm[:, i, :, :] = (X[:, i, :, :] - vmin) / (vmax - vmin + 1e-8)

for i in range(Y.shape[1]):
    vmin, vmax = np.nanmin(Y[:, i, :, :]), np.nanmax(Y[:, i, :, :])
    Y_norm[:, i, :, :] = (Y[:, i, :, :] - vmin) / (vmax - vmin + 1e-8)

print("✅ Normalized all variables for ML training.")


In [ ]:
np.save("/content/drive/MyDrive/ClimateProject/X_2015_2025_4inputs.npy", X_norm)
np.save("/content/drive/MyDrive/ClimateProject/Y_2015_2025_2outputs.npy", Y_norm)
print("✅ Saved ML-ready arrays to Drive.")


In [ ]:
import matplotlib.pyplot as plt

t = 0  # first month
fig, axes = plt.subplots(1, 2, figsize=(10,4))
axes[0].imshow(X_norm[t, 0, :, :], cmap="inferno")
axes[0].set_title("CO₂ (normalized) — Jan 2015")
axes[1].imshow(Y_norm[t, 0, :, :], cmap="coolwarm")
axes[1].set_title("Temperature (normalized) — Jan 2015")
plt.show()


**Stage 7 ML Emulator Training **

In [ ]:
!pip install torch torchvision tqdm -q
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np


In [ ]:
X = np.load("/content/drive/MyDrive/ClimateProject/X_2015_2025_4inputs.npy")
Y = np.load("/content/drive/MyDrive/ClimateProject/Y_2015_2025_2outputs.npy")

print("✅ Loaded data:", X.shape, Y.shape)


In [ ]:
class ClimateDataset(Dataset):
    def __init__(self, X, Y):
        # ensure shape is [time, ch, height, width] = [132, ch, 96, 144]
        if X.shape[-2] != 96:
            X = np.transpose(X, (0, 1, 3, 2))  # swap last two dims
        if Y.shape[-2] != 96:
            Y = np.transpose(Y, (0, 1, 3, 2))
        self.X = torch.tensor(X, dtype=torch.float32)
        self.Y = torch.tensor(Y, dtype=torch.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]


In [ ]:
dataset = ClimateDataset(X, Y)
train_loader = DataLoader(dataset, batch_size=4, shuffle=True)
print("✅ Dataset fixed: X shape", dataset.X.shape, "| Y shape", dataset.Y.shape)


In [ ]:
class SimpleUNet(nn.Module):
    def __init__(self, in_ch=4, out_ch=2):
        super(SimpleUNet, self).__init__()
        self.enc1 = nn.Sequential(nn.Conv2d(in_ch, 32, 3, padding=1), nn.ReLU(), nn.Conv2d(32, 32, 3, padding=1), nn.ReLU())
        self.pool1 = nn.MaxPool2d(2)
        self.enc2 = nn.Sequential(nn.Conv2d(32, 64, 3, padding=1), nn.ReLU())
        self.pool2 = nn.MaxPool2d(2)
        self.middle = nn.Sequential(nn.Conv2d(64, 128, 3, padding=1), nn.ReLU())
        self.up1 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.dec1 = nn.Sequential(nn.Conv2d(128, 64, 3, padding=1), nn.ReLU())
        self.up2 = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=False)
        self.dec2 = nn.Sequential(nn.Conv2d(64, 32, 3, padding=1), nn.ReLU(), nn.Conv2d(32, out_ch, 1))
    def forward(self, x):
        x1 = self.enc1(x)
        x2 = self.pool1(x1)
        x3 = self.enc2(x2)
        x4 = self.pool2(x3)
        x5 = self.middle(x4)
        x6 = self.up1(x5)
        x7 = self.dec1(x6)
        x8 = self.up2(x7)
        out = self.dec2(x8)
        return out


In [ ]:
device = torch.device("cpu")  # CPU mode
model = SimpleUNet(in_ch=4, out_ch=2).to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

print("✅ Model initialized (CPU mode).")


In [ ]:
# Replace NaN or Inf with finite values
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
Y = np.nan_to_num(Y, nan=0.0, posinf=0.0, neginf=0.0)
print("✅ Replaced NaN/Inf with zeros.")


In [ ]:
print("X range:", np.nanmin(X), "→", np.nanmax(X))
print("Y range:", np.nanmin(Y), "→", np.nanmax(Y))


In [ ]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)  # smaller LR

for epoch in range(10):
    model.train()
    total_loss = 0
    for Xb, Yb in tqdm(train_loader, desc=f"Epoch {epoch+1}/10"):
        Xb, Yb = Xb.to(device), Yb.to(device)
        optimizer.zero_grad()
        preds = model(Xb)
        preds = torch.nan_to_num(preds)   # avoid inf/NaN in prediction
        loss = criterion(preds, Yb)
        if torch.isnan(loss):
            print("⚠️ NaN loss detected — skipping batch.")
            continue
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # prevent exploding grads
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: Loss = {total_loss/len(train_loader):.6f}")


**ConvLSTM Climate Emulator (4 Inputs → 2 Outputs, 2015–2025)**

In [ ]:
!pip install torch torchvision tqdm -q
import torch, torch.nn as nn, torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import numpy as np


In [ ]:
X = np.load("/content/drive/MyDrive/ClimateProject/X_2015_2025_4inputs.npy")
Y = np.load("/content/drive/MyDrive/ClimateProject/Y_2015_2025_2outputs.npy")

# fix axis order if needed
if X.shape[-2] != 96: X = np.transpose(X, (0,1,3,2))
if Y.shape[-2] != 96: Y = np.transpose(Y, (0,1,3,2))

print("Data shapes:", X.shape, Y.shape)   # (132, 4, 96, 144), (132, 2, 96, 144)


In [ ]:
class ClimateSeqDataset(Dataset):
    def __init__(self, X, Y, seq_len=6):
        self.seq_len = seq_len
        self.X_seq, self.Y_seq = [], []
        for i in range(len(X)-seq_len):
            self.X_seq.append(X[i:i+seq_len])
            self.Y_seq.append(Y[i+seq_len])
        self.X_seq, self.Y_seq = np.array(self.X_seq), np.array(self.Y_seq)
    def __len__(self): return len(self.X_seq)
    def __getitem__(self, idx):
        return torch.tensor(self.X_seq[idx], dtype=torch.float32), \
               torch.tensor(self.Y_seq[idx], dtype=torch.float32)

dataset = ClimateSeqDataset(X, Y, seq_len=6)
train_loader = DataLoader(dataset, batch_size=2, shuffle=True)
print("✅ Sequence dataset built:", len(dataset), "samples, each 6 months → next month")


In [ ]:
import torch.nn.functional as F

class ConvLSTMCell(nn.Module):
    def __init__(self, in_ch, hidden_ch, kernel_size=3):
        super().__init__()
        pad = kernel_size // 2
        self.hidden_ch = hidden_ch
        self.conv = nn.Conv2d(in_ch + hidden_ch, 4 * hidden_ch, kernel_size, padding=pad)

    def forward(self, x, h, c):
        # Replace any NaNs/Infs with zeros
        x = torch.nan_to_num(x)
        h = torch.nan_to_num(h)
        c = torch.nan_to_num(c)

        combined = torch.cat([x, h], dim=1)
        conv_out = self.conv(combined)
        i, f, o, g = torch.split(conv_out, self.hidden_ch, dim=1)
        i, f, o = torch.sigmoid(i), torch.sigmoid(f), torch.sigmoid(o)
        g = torch.tanh(g)
        c_next = f * c + i * g
        h_next = o * torch.tanh(c_next)

        # Clamp to prevent explosion
        h_next = torch.clamp(h_next, -5, 5)
        c_next = torch.clamp(c_next, -5, 5)

        return h_next, c_next


class ConvLSTMEmulator(nn.Module):
    def __init__(self, in_ch=4, hidden_ch=16, out_ch=2):
        super().__init__()
        self.cell = ConvLSTMCell(in_ch, hidden_ch)
        self.conv_out = nn.Conv2d(hidden_ch, out_ch, 1)

    def forward(self, seq):
        b, t, c, h, w = seq.size()
        h_t = torch.zeros(b, 16, h, w, device=seq.device)
        c_t = torch.zeros_like(h_t)
        for i in range(t):
            h_t, c_t = self.cell(seq[:, i], h_t, c_t)
        out = self.conv_out(h_t)
        return out


In [ ]:
X = np.nan_to_num(X, nan=0.0, posinf=0.0, neginf=0.0)
Y = np.nan_to_num(Y, nan=0.0, posinf=0.0, neginf=0.0)

print("✅ Data cleaned:", np.isnan(X).sum(), "NaNs in X,", np.isnan(Y).sum(), "in Y")


In [ ]:
device = torch.device("cpu")
model = ConvLSTMEmulator().to(device)

# Use a more stable loss
criterion = nn.SmoothL1Loss()   # Huber loss instead of MSE
optimizer = optim.Adam(model.parameters(), lr=5e-4)  # smaller, safer LR

epochs = 10
for epoch in range(epochs):
    model.train()
    total_loss = 0.0
    for Xb, Yb in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs}"):
        Xb, Yb = Xb.to(device), Yb.to(device)

        # Replace NaN / Inf before forward pass
        Xb = torch.nan_to_num(Xb, nan=0.0, posinf=0.0, neginf=0.0)
        Yb = torch.nan_to_num(Yb, nan=0.0, posinf=0.0, neginf=0.0)

        optimizer.zero_grad()
        preds = model(Xb)

        # Replace any NaN predictions
        preds = torch.nan_to_num(preds, nan=0.0, posinf=0.0, neginf=0.0)

        loss = criterion(preds, Yb)

        # Skip if loss is NaN
        if torch.isnan(loss):
            print("⚠️ NaN loss detected — skipping batch.")
            continue

        loss.backward()

        # Prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()
        total_loss += loss.item()
    avg_loss = total_loss / len(train_loader)
    rmse = (avg_loss ** 0.5)
    print(f"Epoch {epoch+1}: Loss = {avg_loss:.6f} | RMSE = {rmse:.6f}")




print("✅ Training finished successfully.")


**RMSE VISUALIZATION**

In [ ]:
import torch
import numpy as np

model.eval()
preds_all = []
trues_all = []

with torch.no_grad():
    for Xb, Yb in train_loader:
        Xb, Yb = Xb.to(device), Yb.to(device)
        pred = model(Xb)
        preds_all.append(pred.cpu().numpy())
        trues_all.append(Yb.cpu().numpy())

preds_all = np.concatenate(preds_all, axis=0)
trues_all = np.concatenate(trues_all, axis=0)

print("✅ Predictions collected:", preds_all.shape, trues_all.shape)


In [ ]:
rmse_map = np.sqrt(np.mean((preds_all - trues_all) ** 2, axis=0))
print("RMSE map shape:", rmse_map.shape)


In [ ]:
import matplotlib.pyplot as plt

titles = ["Temperature RMSE (°C)", "Precipitation RMSE (mm/day)"]

fig, axes = plt.subplots(1, 2, figsize=(12, 4))
for i, ax in enumerate(axes):
    im = ax.imshow(rmse_map[i], cmap="magma", origin="lower")
    ax.set_title(titles[i])
    plt.colorbar(im, ax=ax, shrink=0.7)
plt.suptitle("🌎 Spatial RMSE Maps — ConvLSTM Emulator (2015–2025)", fontsize=14)
plt.show()


In [ ]:
rmse_norm = rmse_map / np.max(rmse_map, axis=(1,2), keepdims=True)


In [ ]:
print("🌡️  Mean RMSE (Temperature):", np.mean(rmse_map[0]))
print("🌧️  Mean RMSE (Precipitation):", np.mean(rmse_map[1]))


**Real Earth Projection**

In [ ]:
rmse_map.shape = (2, 96, 144)


In [ ]:
!pip install cartopy -q


In [ ]:
import numpy as np

# Regular 2.5° grid (since 144×96 = 360/2.5 × 180/2.5)
lons = np.linspace(-180, 180, 144)
lats = np.linspace(-90, 90, 96)
lon2d, lat2d = np.meshgrid(lons, lats)

print("Latitude range:", lats[0], "→", lats[-1])
print("Longitude range:", lons[0], "→", lons[-1])


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

titles = ["Temperature RMSE (°C)", "Precipitation RMSE (mm/day)"]

fig = plt.figure(figsize=(14, 5))
for i in range(2):
    ax = plt.subplot(1, 2, i + 1, projection=ccrs.PlateCarree())
    ax.set_global()
    im = ax.pcolormesh(lon2d, lat2d, rmse_map[i],
                       cmap="magma", shading="auto", transform=ccrs.PlateCarree())
    ax.coastlines(linewidth=0.8)
    ax.add_feature(cfeature.BORDERS, linewidth=0.3)
    ax.set_title(titles[i], fontsize=12)
    plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
plt.suptitle("🌎 Spatial RMSE Maps — ConvLSTM Emulator (NorESM2-LM, 2015–2025)", fontsize=14)
plt.tight_layout()
plt.show()


In [ ]:
rmse_norm = rmse_map / np.nanmax(rmse_map, axis=(1,2), keepdims=True)

fig = plt.figure(figsize=(14, 5))
for i in range(2):
    ax = plt.subplot(1, 2, i + 1, projection=ccrs.PlateCarree())
    ax.set_global()
    im = ax.pcolormesh(lon2d, lat2d, rmse_norm[i], cmap="plasma", transform=ccrs.PlateCarree())
    ax.coastlines(linewidth=0.8)
    ax.set_title(f"{titles[i]} (Normalized 0–1)")
    plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05, shrink=0.8)
plt.suptitle("🌎 Normalized RMSE Comparison", fontsize=14)
plt.show()


In [ ]:
print(f"🌡️ Global mean Temperature RMSE: {np.mean(rmse_map[0]):.3f} °C")
print(f"🌧️ Global mean Precipitation RMSE: {np.mean(rmse_map[1]):.3f} mm/day")


**Saving the maps for report**

In [ ]:
import xarray as xr

# make sure lats/lons and rmse_map are defined
ds_rmse = xr.Dataset(
    data_vars={
        "Temperature_RMSE": (("lat", "lon"), rmse_map[0]),
        "Precipitation_RMSE": (("lat", "lon"), rmse_map[1]),
    },
    coords={
        "lat": lats,
        "lon": lons
    },
    attrs={
        "source_model": "NorESM2-LM ConvLSTM Emulator",
        "years": "2015–2025",
        "description": "Spatial RMSE of ConvLSTM emulator (TAS & PR)",
        "units": "°C (TAS), mm/day (PR)"
    }
)

save_path_nc = "/content/drive/MyDrive/ClimateProject/RMSE_maps_2015_2025.nc"
ds_rmse.to_netcdf(save_path_nc)
print(f"✅ RMSE maps saved as NetCDF: {save_path_nc}")


In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature

titles = ["Temperature RMSE (°C)", "Precipitation RMSE (mm/day)"]
file_names = ["Temperature_RMSE.png", "Precipitation_RMSE.png"]

for i in range(2):
    fig = plt.figure(figsize=(8, 4))
    ax = plt.subplot(1, 1, 1, projection=ccrs.PlateCarree())
    ax.set_global()
    im = ax.pcolormesh(lon2d, lat2d, rmse_map[i],
                       cmap="magma", shading="auto", transform=ccrs.PlateCarree())
    ax.coastlines(linewidth=0.8)
    ax.add_feature(cfeature.BORDERS, linewidth=0.3)
    ax.set_title(f"{titles[i]} (2015–2025)", fontsize=12)
    plt.colorbar(im, ax=ax, orientation="horizontal", pad=0.05, shrink=0.8)
    plt.tight_layout()

    save_path_png = f"/content/drive/MyDrive/ClimateProject/{file_names[i]}"
    plt.savefig(save_path_png, dpi=300, bbox_inches="tight")
    plt.close()
    print(f"✅ Saved: {save_path_png}")


In [ ]:
# Reload & check from file
check = xr.open_dataset(save_path_nc)
print(check)


**Plot showing diff between actual and predicted**

*1)Simple line plot*

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# preds_all, trues_all shapes: [samples, 2, lat, lon]
# Take spatial mean for each timestep
true_mean = np.mean(trues_all, axis=(2,3))
pred_mean = np.mean(preds_all, axis=(2,3))

months = np.arange(len(true_mean))  # each step = 1 month
variables = ["Temperature (°C)", "Precipitation (mm/day)"]

plt.figure(figsize=(10,4))
for i in range(2):
    plt.subplot(1,2,i+1)
    plt.plot(months, true_mean[:, i], label="Actual", color='black')
    plt.plot(months, pred_mean[:, i], label="Predicted", color='orange', linestyle='--')
    plt.title(variables[i])
    plt.xlabel("Months since Jan 2015")
    plt.ylabel(variables[i])
    plt.legend()
plt.tight_layout()
plt.show()


*2)Scatter plot*

In [ ]:
plt.figure(figsize=(10,4))
for i, var in enumerate(["Temperature", "Precipitation"]):
    plt.subplot(1,2,i+1)
    plt.scatter(trues_all[:, i, :, :].flatten(),
                preds_all[:, i, :, :].flatten(),
                s=1, alpha=0.3)
    plt.plot([-1,1],[ -1,1 ], 'r--')  # reference diagonal
    plt.title(f"{var}: Predicted vs Actual")
    plt.xlabel("Actual")
    plt.ylabel("Predicted")
plt.tight_layout()
plt.show()


3)Spatial map Prediction Error

In [ ]:
import cartopy.crs as ccrs
import cartopy.feature as cfeature

mean_error = np.mean(preds_all - trues_all, axis=0)  # [2, lat, lon]

titles = ["Temperature Mean Error (°C)", "Precipitation Mean Error (mm/day)"]
fig = plt.figure(figsize=(14,5))
for i in range(2):
    ax = plt.subplot(1,2,i+1, projection=ccrs.PlateCarree())
    im = ax.pcolormesh(lon2d, lat2d, mean_error[i], cmap="coolwarm", transform=ccrs.PlateCarree())
    ax.coastlines(); ax.add_feature(cfeature.BORDERS, linewidth=0.5)
    ax.set_title(titles[i])
    plt.colorbar(im, ax=ax, orientation='horizontal', pad=0.05)
plt.suptitle("🌍 Mean Prediction Error (Predicted − Actual)", fontsize=14)
plt.show()


*4)Histogram of error Distributiom*

In [ ]:
plt.figure(figsize=(10,4))
for i, var in enumerate(["Temperature", "Precipitation"]):
    plt.subplot(1,2,i+1)
    err = (preds_all[:, i, :, :] - trues_all[:, i, :, :]).flatten()
    plt.hist(err, bins=50, color='skyblue', edgecolor='black')
    plt.title(f"{var} Error Distribution")
    plt.xlabel("Predicted − Actual")
    plt.ylabel("Frequency")
plt.tight_layout()
plt.show()


*5)Animated*

In [ ]:
preds_all.shape, trues_all.shape  # => (time, 2, lat, lon)


In [ ]:
import numpy as np

lons = np.linspace(-180, 180, 144)
lats = np.linspace(-90, 90, 96)
lon2d, lat2d = np.meshgrid(lons, lats)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import cartopy.crs as ccrs
import cartopy.feature as cfeature

# Choose variable index (0 = Temperature, 1 = Precipitation)
var_index = 0
var_name = ["Temperature (°C)", "Precipitation (mm/day)"][var_index]

# Compute difference
diff = preds_all[:, var_index] - trues_all[:, var_index]

# Number of frames (limit for Colab speed)
frames = min(len(diff), 120)  # animate first 10 years or so

# Create figure
fig = plt.figure(figsize=(12, 5))
proj = ccrs.PlateCarree()

# Axes: Actual, Predicted, Difference
ax1 = plt.subplot(1, 3, 1, projection=proj)
ax2 = plt.subplot(1, 3, 2, projection=proj)
ax3 = plt.subplot(1, 3, 3, projection=proj)

# Initialize plots
im1 = ax1.pcolormesh(lon2d, lat2d, trues_all[0, var_index], cmap="coolwarm", transform=proj)
im2 = ax2.pcolormesh(lon2d, lat2d, preds_all[0, var_index], cmap="coolwarm", transform=proj)
im3 = ax3.pcolormesh(lon2d, lat2d, diff[0], cmap="bwr", transform=proj)

# Coastlines and titles
for ax, title in zip([ax1, ax2, ax3],
                     [f"Actual {var_name}",
                      f"Predicted {var_name}",
                      "Predicted − Actual"]):
    ax.coastlines()
    ax.add_feature(cfeature.BORDERS, linewidth=0.3)
    ax.set_global()
    ax.set_title(title, fontsize=10)

plt.tight_layout()

# Update function for animation
def update(frame):
    im1.set_array(trues_all[frame, var_index].ravel())
    im2.set_array(preds_all[frame, var_index].ravel())
    im3.set_array(diff[frame].ravel())
    fig.suptitle(f"{var_name} — Month #{frame+1}", fontsize=12)
    return [im1, im2, im3]

# Create animation
ani = animation.FuncAnimation(fig, update, frames=frames, interval=200, blit=False)

# Display animation inline (works on Colab)
from IPython.display import HTML
HTML(ani.to_jshtml())
